In [1]:
DEBUG = False

MODEL_NAME = "rinna/youri-7b-instruction"
MODEL_BASE_NAME = MODEL_NAME.split("/")[-1]
LORA_DIR = f"./pretrained_lora_{MODEL_BASE_NAME}"

OUTPUT_MERGED_DIR = f"./pretrained_merged_{MODEL_BASE_NAME}"
OUTPUT_QUANTIZED_DIR = f"./pretrained_quantized_{MODEL_BASE_NAME}"

In [3]:
from peft import PeftModel  # type: ignore
from transformers import AutoTokenizer, AutoModelForCausalLM
import os

if not os.path.exists(OUTPUT_MERGED_DIR):
    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
    )
    model = PeftModel.from_pretrained(base_model, LORA_DIR)
    model = model.merge_and_unload().half()
    model.save_pretrained(OUTPUT_MERGED_DIR)
    del model  # unload
    del base_model  # unload
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    # save to OUTPUT_SAVE_DIR
    tokenizer.save_pretrained(OUTPUT_MERGED_DIR)
else:
    tokenizer = AutoTokenizer.from_pretrained(OUTPUT_MERGED_DIR)

In [4]:
example_text = """
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
手術によって天才となったチャーリイ・ゴードンの日記という形式を通して語られる、ダニエル・キイスの小説は何?

### 入力: 
- ダニエル・キイス 「アルジャーノンに花束を」は中編もそれを発展させた長編も、知的障害者のチャーリイが実験的手術によって天才となり、短期間でその効果が消えてしまった経過を報告書形式で書いたものである。中編が1959年4月号のF&SF誌に掲載され、1966年に長編化された。この小説は何度か他のメディアに採用されており、1968年の映画『まごころを君に』(出演: クリフ・ロバートソン、クレア・ブルーム)、2002年の日本のテレビドラマ『アルジャーノンに花束を』(出演: ユースケ・サンタマリア、菅野美穂)、などがある。クリフ・ロバートソンはこの作品でアカデミー主演男優賞を受賞している。また、キイスは中編版で1959年にヒューゴー賞、長編版で1966年にネビュラ賞を受賞した。また2015年にも日本のテレビドラマ『アルジャーノンに花束を』(出演:山下智久、窪田正孝など)で新版長編も放送された。
- ダニエル・キイス ニューヨーク州ブルックリン区生まれ。17歳のとき U.S. Maritime Service に就職し船員となった。その後しばらくして、働きながら心理学や文学に興味を抱くようになりブルックリンカレッジで心理学の学士号を取得し、一時期ファッション写真のスタジオで働き、その後ニューヨークの高校で国語教師を務めつつ、定時制で英米文学を学び、週末には小説を書いていた。最終的に英米文学の修士号を得ている。1950年代初め、パルプ雑誌『マーヴェル・サイエンス・ストーリーズ』の編集者となった。この雑誌の出版者であるマーティン・グッドマン(英語版)はアメリカン・コミックスも手がけており、マーベル・コミックの前身となったタイムリー・コミック(1940年代)やアトラス・コミック(1950年代)を出版していた。
- アルジャーノンに花束を 『アルジャーノンに花束を』(アルジャーノンにはなたばを、Flowers for Algernon)は、アメリカ合衆国の作家ダニエル・キイスによるSF小説。1959年に中編小説として発表され、翌年ヒューゴー賞短編小説部門を受賞。1966年に長編小説として改作され、ネビュラ賞を受賞した。それまでのSF小説が宇宙や未来などを舞台とした作品であったのに比べ、本作は知能指数を高める手術とそれに付随する事柄という限定した範囲での前提でSFとして成立させている。ジュディス・メリルは、本作をSFの多様性をあらわす作品のひとつとして位置づけている。また、最後の一文が主眼であり、ここに収束される感動に泣かされる作品でもある。

### 応答: 
"""

examples = [tokenizer(example_text)]

In [5]:
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig  # type: ignore

quantize_config = BaseQuantizeConfig(
    bits=4,  # quantize model to 4-bit
    group_size=128,  # it is recommended to set the value to 128
    desc_act=False,  # set to False can significantly speed up inference but the perplexity may slightly bad
)
if not os.path.exists(OUTPUT_QUANTIZED_DIR):
    model = AutoGPTQForCausalLM.from_pretrained(OUTPUT_MERGED_DIR, quantize_config)
    model.quantize(examples)
    model.save_quantized(OUTPUT_QUANTIZED_DIR, use_safetensors=True)
    tokenizer.save_pretrained(OUTPUT_QUANTIZED_DIR)

Loading checkpoint shards: 100%|██████████| 3/3 [00:19<00:00,  6.54s/it]


In [7]:
import torch
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoGPTQForCausalLM.from_quantized(
    OUTPUT_QUANTIZED_DIR, device="cuda:0", use_safetensors=True
)

Skipping module injection for FusedLlamaMLPForQuantizedModel as currently not supported with use_triton=False.


In [8]:
RESPONSE_MESSAGE = "応答"
RESPONSE_PROMPT = f"### {RESPONSE_MESSAGE}: \n"


def build_prompt(
    user_message: str,
    inputs: str | None = "",
    separator: str = "\n\n### ",
    response_message: str = RESPONSE_MESSAGE,
) -> str:
    system_message = "以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。"
    prompt = system_message
    roles = ["指示", response_message]
    messages = [": \n" + user_message, ": \n"]

    if inputs:
        roles = ["指示", "入力", response_message]
        messages = [": \n" + user_message, ": \n" + inputs, ": \n"]

    for role, message in zip(roles, messages):
        prompt += separator + role + message
    return prompt

In [9]:
import pandas as pd
import datasets

ds = datasets.load_dataset("hotchpotch/jaqket_v1_qa_wikija_context")  # type: ignore
valid_ds = ds["validation"]  # type: ignore
valid_df = valid_ds.data.to_pandas()  # type: ignore
# context は list なので、 "\n" で結合する
valid_df["context"] = valid_df["context"].apply(lambda x: "\n".join(x) + "\n")
valid_df.head(1)

,qid,question,answer,context,answers,competition,timestamp,section,number,original_question,original_answer,original_additional_info
0,QA20QBIK-0912,手術によって天才となったチャーリイ・ゴードンの日記という形式を通して語られる、ダニエル・キイ...,アルジャーノンに花束を,ダニエル・キイス 「アルジャーノンに花束を」は中編もそれを発展させた長編も、知的障害者のチャ...,[アルジャーノンに花束を],第1回AI王,2020/01/27,開発データ問題 (dev1),912,手術によって天才となったチャーリイ・ゴードンの日記という形式を通して語られる、ダニエル・キイ...,アルジャーノンに花束を,


In [10]:
import torch


def qa(model, tokenizer, question, context, build_prompt_fn=build_prompt):
    prompt = build_prompt_fn(question, context)
    tokenizer.pad_token = tokenizer.eos_token
    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    with torch.no_grad():
        output_ids = model.generate(
            input_ids=token_ids.to(model.device),  # type: ignore
            max_new_tokens=24,
            do_sample=True,
            temperature=0.8,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    output = tokenizer.decode(output_ids.tolist()[0])
    # prompt を取り除く
    output = output.replace(prompt, "")
    # eos_token 以降を取り除く
    output = output.split(tokenizer.eos_token)[0]
    return output.strip()


target_n = 0

print(
    qa(
        model,
        tokenizer,
        valid_df["question"][target_n],
        valid_df["context"][target_n],
        build_prompt_fn=build_prompt,
    )
)

アルジャーノンに花束を


In [11]:
# valid_df での正解率を測る
from tqdm import tqdm

valid_df = valid_df.reset_index(drop=True)
for i in tqdm(range(len(valid_df))):
    question = valid_df["question"][i]
    context = valid_df["context"][i]
    answer = valid_df["answer"][i]
    pred = qa(model, tokenizer, question, context, build_prompt_fn=build_prompt)
    valid_df.loc[i, "pred"] = pred

100%|██████████| 980/980 [03:34<00:00,  4.56it/s]


In [12]:
# 完全一致の正解率を表示
valid_df["is_correct"] = valid_df["answer"] == valid_df["pred"]
valid_df["is_correct"].mean()

0.726530612244898

In [13]:
# 間違ったものだけを表示
valid_df[["question", "answer", "pred", "is_correct"]][valid_df["is_correct"] == False]

,question,answer,pred,is_correct
0,手術によって天才となったチャーリイ・ゴードンの日記という形式を通して語られる、ダニエル・キイ...,アルジャーノンに花束を,アルジャーノンに花束を』,False
3,現代の「世界三大ギタリスト」と称されるのはデレク・トラックス、ジョン・フルシアンテと誰?,ジョン・メイヤー,ジョン・フルシアンテ,False
4,テレビ番組の検証映像でもよく使用される、映した物体の温度を色分けして表示する装置を何というで...,サーモグラフィー,色温度計,False
5,不景気にもかかわらず、原材料の高騰などで物価水準が上昇し続ける状態のことを経済用語で何という?,スタグフレーション,デフレーション,False
7,古代ギリシャの彫刻が起源とされ、飛鳥時代の仏像にも見られる、口元に微笑みを浮かべたように見え...,アルカイク・スマイル,アルカイック・スマイル,False
...,...,...,...,...
964,旅客機の座席で、ファーストクラスとエコノミークラスの中間にあたるものを一般に何クラスというで...,ビジネスクラス,プレミアム・エコノミークラス,False
966,2020年シーズン、読売ジャイアンツの丸佳浩が試合での登場時に主に流していたのは、何という歌...,LiSA,Rising Hope』,False
967,2人で行う花札の遊び方で、「五光」や「猪鹿蝶」といった役があるのは何?,こいこい,花合わせ,False
971,アンデルセンの『人魚姫』を題材にした、1989年公開のディズニー映画は何でしょう?,リトル・マーメイド,リトル・マーメイド』,False


In [14]:
# 部分一致の正解率を表示
valid_df["is_correct"] = valid_df.apply(lambda x: x["answer"] in x["pred"], axis=1)
valid_df["is_correct"].mean()

0.8673469387755102

In [15]:
# 間違ったものだけを表示
valid_df[["question", "answer", "pred", "is_correct"]][valid_df["is_correct"] == False]

,question,answer,pred,is_correct
3,現代の「世界三大ギタリスト」と称されるのはデレク・トラックス、ジョン・フルシアンテと誰?,ジョン・メイヤー,ジョン・フルシアンテ,False
4,テレビ番組の検証映像でもよく使用される、映した物体の温度を色分けして表示する装置を何というで...,サーモグラフィー,色温度計,False
5,不景気にもかかわらず、原材料の高騰などで物価水準が上昇し続ける状態のことを経済用語で何という?,スタグフレーション,デフレーション,False
7,古代ギリシャの彫刻が起源とされ、飛鳥時代の仏像にも見られる、口元に微笑みを浮かべたように見え...,アルカイク・スマイル,アルカイック・スマイル,False
9,幕末から明治にかけて活躍した佐賀藩士で、日本赤十字社を創設したのは誰?,佐野常民,赤十字社創始者 大給恒,False
...,...,...,...,...
951,日本語では「工場制手工業」という、機械制大工業が出現する以前に行われていた最初の資本主義的生...,マニュファクチュア,工場制手工業,False
964,旅客機の座席で、ファーストクラスとエコノミークラスの中間にあたるものを一般に何クラスというで...,ビジネスクラス,プレミアム・エコノミークラス,False
966,2020年シーズン、読売ジャイアンツの丸佳浩が試合での登場時に主に流していたのは、何という歌...,LiSA,Rising Hope』,False
967,2人で行う花札の遊び方で、「五光」や「猪鹿蝶」といった役があるのは何?,こいこい,花合わせ,False


: 